In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss,accuracy_score
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet

In [2]:
insurance=pd.read_csv("insurance.csv")
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
# insurance = pd.get_dummies(insurance,dtype = int,drop_first = True)

In [4]:
lbl=LabelEncoder()
insurance['sex']=lbl.fit_transform(insurance['sex'])
insurance['smoker']=lbl.fit_transform(insurance['smoker'])
insurance['region']=lbl.fit_transform(insurance['region'])

In [5]:
X = insurance.drop("charges",axis =1)
y = insurance['charges']

In [6]:
elastic = ElasticNet()
kfold = KFold(n_splits=5,shuffle=True,random_state=23)

In [7]:
alpha = np.linspace(0.001,0.999,20)
l1_ratio = np.linspace(0.001,4,20)

In [8]:
params = {'alpha':alpha,'l1_ratio':l1_ratio}

In [9]:
scaler  = StandardScaler()
pipe = Pipeline([('SCL',scaler),('elastic',elastic)])

In [10]:
params = {'elastic__alpha':alpha,'elastic__l1_ratio':l1_ratio}

In [12]:
gcv = GridSearchCV(pipe,param_grid=params,cv=kfold,scoring="r2")
gcv.fit(X,y)

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
1500 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 427, in fit


GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('elastic', ElasticNet())]),
             param_grid={'elastic__alpha': array([0.001     , 0.05352632, 0.10605263, 0.15857895, 0.21110526,
       0.26363158, 0.31615789, 0.36868421, 0.42121053, 0.47373684,
       0.52626316, 0.57878947, 0.63131579, 0.68384211, 0.73636842,
       0.78889474, 0.....999     ]),
                         'elastic__l1_ratio': array([1.00000000e-03, 2.11473684e-01, 4.21947368e-01, 6.32421053e-01,
       8.42894737e-01, 1.05336842e+00, 1.26384211e+00, 1.47431579e+00,
       1.68478947e+00, 1.89526316e+00, 2.10573684e+00, 2.31621053e+00,
       2.52668421e+00, 2.73715789e+00, 2.94763158e+00, 3.15810526e+00,
       3.36857895e+00, 3.57905263e+00, 3.78952632e+00, 4.00000000e+00])},
             scoring='r2')

In [13]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'elastic__alpha': 0.001, 'elastic__l1_ratio': 0.001}
best score : 0.7475669584177035


In [14]:
knn = KNeighborsRegressor()
kfold = KFold(n_splits=5,shuffle=True,random_state=23)
pipe=Pipeline([('SCL',scaler),('KNN',knn)])
params={'KNN__n_neighbors':np.arange(1,11)}

In [15]:
gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='r2')
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('KNN', KNeighborsRegressor())]),
             param_grid={'KNN__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             scoring='r2')

In [16]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'KNN__n_neighbors': 9}
best score : 0.8191986032654075
